# Label resolver development

In [1]:
##################################################################################
# Imports
##################################################################################
# scikit learn imports
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.preprocessing import scale, StandardScaler, Normalizer, label_binarize
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_recall_curve, accuracy_score, f1_score, precision_score, recall_score, classification_report, roc_curve, auc, roc_auc_score, confusion_matrix
from sklearn.model_selection import PredefinedSplit
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import LinearSVC

# visualization
import seaborn as sns
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.colors
from mpl_toolkits.mplot3d import Axes3D

# Natural Language Toolkit
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.corpus import stopwords
#nltk.download()
from nltk import ngrams, pos_tag
import spacy
import re

import numpy as np
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
import time
import argparse
import pdb
import random
import collections, numpy
import json
import pandas as pd
import os
import glob
import itertools
import operator

# saving model
import shutil

# Visualization tools
from tensorboardX import SummaryWriter
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# saving the best models
saved_models = []
import pickle

2021-10-25 11:56:35.723953: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [13]:
word2label_file = '/home/anjani/distant-PICO/CandidateGeneration/ResultInspection/resolve_annot_corpus.tsv'

In [14]:
data = pd.read_csv(word2label_file, sep='\t', error_bad_lines=False, header=None)
print('Shape of the data: ', data.shape)
data.head()

Shape of the data:  (1973468, 3)


,0,1,2
0,0,Infant,1
1,1,Child,1
2,2,Retinoblastoma,1
3,3,25,1
4,4,carboplatin,2


In [15]:
X, X_test, y, y_test = train_test_split(data[1], data[2], test_size=0.05, train_size=0.95, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.20,train_size =0.80)

In [16]:
def my_tokenizer(doc):
    tokens = doc.split(' ')
    return tokens

In [17]:
REPLACE_BY_SPACE_RE = re.compile('[!@#$%^&*()[]{};:,./<>?\|`~=_+]')
STOPWORDS = set(stopwords.words('english'))

def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = REPLACE_BY_SPACE_RE.sub(' ', str(text))# replace REPLACE_BY_SPACE_RE symbols by space in text
    text = ' '.join([w for w in text.split() if not w in STOPWORDS])# delete stopwords from text
    return text.lower()

X_train = [text_prepare(x) for x in X_train]
X_val = [text_prepare(x) for x in X_val]

In [18]:
# Dictionary of all words from train corpus with their counts.
words_counts = {}
for comments in X_train:
    for word in str(comments).split():
        if word not in words_counts:
            words_counts[word] = 1
        words_counts[word] += 1
        
DICT_SIZE = 30000
POPULAR_WORDS = sorted(words_counts, key=words_counts.get, reverse=True)[:DICT_SIZE]
WORDS_TO_INDEX = {key: rank for rank, key in enumerate(POPULAR_WORDS, 0)}
INDEX_TO_WORDS = {index:word for word, index in WORDS_TO_INDEX.items()}
ALL_WORDS = WORDS_TO_INDEX.keys()

In [19]:
def my_bag_of_words(text, words_to_index, dict_size):
    """
        text: a string
        dict_size: size of the dictionary
        
        return a vector which is a bag-of-words representation of 'text'
    """
    result_vector = np.zeros(dict_size)
    
    for word in text.split(' '):
        if word in words_to_index:
            result_vector[words_to_index[word]] +=1
    return result_vector

#X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
#X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
#print('X_train shape ', X_train_mybag.shape, '\nX_val shape ', X_val_mybag.shape)

In [20]:
def train_classifier(X_train, y_train, C, regularisation):
    """
      X_train, y_train — training data
      
      return: trained classifier
    """
    count_vectorizer = CountVectorizer(preprocessor=text_prepare, tokenizer=my_tokenizer, vocabulary=WORDS_TO_INDEX, max_features=DICT_SIZE)
    # Create and fit LogisticRegression wraped into OneVsRestClassifier.
    model = OneVsRestClassifier(LogisticRegression(penalty=regularisation, C=C, max_iter=10000))
    
    log_reg_clf_bow = Pipeline([('vect', count_vectorizer), ('clf', model)])
    
    return log_reg_clf_bow

model_pipeline = train_classifier(X_train, y_train, C = 4, regularisation = 'l2')
classifier_mybag = model_pipeline.fit(X_train, y_train)

y_val_predicted_labels_mybag = classifier_mybag.predict(X_val)
y_val_predicted_labels_mybag_proba = classifier_mybag.predict_proba(X_val)

In [21]:
def print_evaluation_scores(y_test, predicted):
    
    print('Accuracy: ', accuracy_score(y_test, predicted, normalize=True))
    print('F1-score macro: ', f1_score(y_test, predicted, average='macro'))
    print('F1-score micro: ', f1_score(y_test, predicted, average='micro'))
    print('F1-score weighted: ', f1_score(y_test, predicted, average='weighted'))
    print(metrics.classification_report(y_test, predicted))
    
print('Bag-of-words\n')
print_evaluation_scores(y_val, y_val_predicted_labels_mybag)

Bag-of-words

Accuracy:  0.9382199120437168
F1-score macro:  0.940638377762026
F1-score micro:  0.9382199120437168
F1-score weighted:  0.9380615127875906
              precision    recall  f1-score   support

           1       0.96      0.96      0.96    178819
           2       0.91      0.95      0.93     97837
           3       0.90      0.86      0.88     71404
           4       1.00      1.00      1.00     26899

    accuracy                           0.94    374959
   macro avg       0.94      0.94      0.94    374959
weighted avg       0.94      0.94      0.94    374959



In [22]:
from joblib import dump, load
dump(classifier_mybag, '/home/anjani/distant-PICO/CandidateGeneration/AnnotationAggregation/resolver_models/resolver_large.joblib') 

['/home/anjani/distant-PICO/CandidateGeneration/AnnotationAggregation/resolver_models/resolver_large.joblib']

## Evaluation (using manually labeled test set)

In [ ]:
# TODO: X_test and y_test need to be manually annotated and should later be used for evaluation of this pipeline

In [23]:
eval_file = '/home/anjani/distant-PICO/CandidateGeneration/ResultInspection/resolve_annot_corpus_eval.tsv'

In [24]:
with open(eval_file, 'w+') as ef:
    X_test.to_csv(ef, sep='\t')